In [1]:
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt
import geopandas
import censusdata
import numpy as np
import seaborn as sns
from census import Census
import us

In [2]:
# panda table setting 
# https://re-thought.com/how-to-suppress-scientific-notation-in-pandas/
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.precision', 2)

# set float display
pd.options.display.float_format = '{:.0f}'.format

In [3]:
 path = r'/Users/bowenli/Documents/GitHub/Data-II-Project'
# path = r'C:/Users/ShrekTheOger/Documents/GitHub/Data-II-Project'
#path = r'C:\Users\engel\Documents\GitHub\Data-II-Project'

## Getting population data of Cook County

In [4]:
col_list = ['Year', 'id', 'Geographic Area Name', 'Estimate!!Total',
            'Estimate!!Total!!Native',
            'Estimate!!Total!!Native!!Born in state of residence',
            'Estimate!!Total!!Native!!Born in other state in the United States',
            'Estimate!!Total!!Native!!Born outside the United States',
            'Estimate!!Total!!Foreign born',
            'Estimate!!Total!!Foreign born!!Naturalized U.S. citizen',
            'Estimate!!Total!!Foreign born!!Not a U.S. citizen']

In [5]:
def pop_data_retrive(y):
    pop = os.path.join(path,'productDownload_2021-11-13T114325',f'ACSDT5Y{y}.B05002_data_with_overlays_2021-11-13T114317.csv')
    df = pd.read_csv(pop)
    return df

In [6]:
def pop_df_clean(df,y):
    df.columns = df.iloc[0]
    df.drop(index = [0,1], inplace = True)
    df = df.loc[:, ~df.columns.str.contains('Error', case = False)]
    df.insert(0,'Year',y)
    df =df[col_list]
    return df

In [7]:
def pop_df_concat(df,pop_df):
    pop_df = pop_df.append(df)
    return pop_df

In [8]:
def pop_df_go():
    years = list(range(2010,2020))
    pop_df = pd.DataFrame()
    for i in years:
        if i != 2019:
            df = pop_data_retrive(i)
            df = pop_df_clean(df,i)
            pop_df = pop_df_concat(pop_df,df)
        else:
            df = pop_data_retrive(i)
            df.loc[0,:] = [i.replace(':', '') for i in list(df.loc[0])]
            df
            df = pop_df_clean(df,i)
            pop_df = pop_df_concat(pop_df,df)
    return pop_df

In [9]:
pop_df_go()

,Year,id,Geographic Area Name,Estimate!!Total,Estimate!!Total!!Native,Estimate!!Total!!Native!!Born in state of residence,Estimate!!Total!!Native!!Born in other state in the United States,Estimate!!Total!!Native!!Born outside the United States,Estimate!!Total!!Foreign born,Estimate!!Total!!Foreign born!!Naturalized U.S. citizen,Estimate!!Total!!Foreign born!!Not a U.S. citizen
2,2019,0500000US17031,"Cook County, Illinois",5198275,4099447,3198804,819189,81454,1098828,556813,542015
2,2018,0500000US17031,"Cook County, Illinois",5223719,4124060,3217517,824977,81566,1099659,550515,549144
2,2017,0500000US17031,"Cook County, Illinois",5238541,4133162,3230676,822928,79558,1105379,544484,560895
2,2016,0500000US17031,"Cook County, Illinois",5227575,4125845,3222677,824586,78582,1101730,534110,567620
2,2015,0500000US17031,"Cook County, Illinois",5236393,4127954,3224748,825134,78072,1108439,529912,578527
2,2014,0500000US17031,"Cook County, Illinois",5227827,4126072,3220139,828693,77240,1101755,518184,583571
2,2013,0500000US17031,"Cook County, Illinois",5212372,4113454,3205604,832394,75456,1098918,506222,592696
2,2012,0500000US17031,"Cook County, Illinois",5197677,4098252,3186629,836550,75073,1099425,500383,599042
2,2011,0500000US17031,"Cook County, Illinois",5182969,4092973,3175519,844343,73111,1089996,491133,598863
2,2010,0500000US17031,"Cook County, Illinois",5172848,4085967,3168508,847240,70219,1086881,485662,601219


In [10]:
index_2010= ['Year','id','Geographic Area Name',
             'Households!!Estimate!!Total',
             'Households!!Estimate!!Less than $10,000',
             'Households!!Estimate!!$10,000 to $14,999',
             'Households!!Estimate!!$15,000 to $24,999',
             'Households!!Estimate!!$25,000 to $34,999',
             'Households!!Estimate!!$35,000 to $49,999',
             'Households!!Estimate!!$50,000 to $74,999',
             'Households!!Estimate!!$75,000 to $99,999',
             'Households!!Estimate!!$100,000 to $149,999',
             'Households!!Estimate!!$150,000 to $199,999',
             'Households!!Estimate!!$200,000 or more',
             'Households!!Estimate!!Median income (dollars)',
             'Households!!Estimate!!Mean income (dollars)']

In [11]:
def income_data_retrive(y):
    pop = os.path.join(path,'productDownload_2021-11-15T220401',f'ACSST1Y{y}.S1901_data_with_overlays_2021-11-15T220349.csv')
    df = pd.read_csv(pop)
    return df

In [12]:
def income_df_clean(df,y):
    df.columns = df.iloc[0]
    df.drop(index = [0], inplace = True)
    df = df.loc[:, ~df.columns.str.contains('Error', case = False)]
    df.insert(0,'Year',y)
    df = df.iloc[:,:16]
    df = df.T
    df.columns = ["%s-%s"%(df.iloc[0,0],df.iloc[2,0]),"%s-%s"%(df.iloc[0,1],df.iloc[2,1][:11])]
    df.index.name = None
    index_name = df.index.tolist()
    index_lib = dict(zip(index_name, index_2010))
    df.rename(index = index_lib, inplace = True)
    return df

In [13]:
def income_df_go():
    years = list(range(2010,2020))
    income_df = pd.DataFrame()
    for i in years:
        df = income_data_retrive(i)
        df = income_df_clean(df,i)
        income_df = pd.concat([income_df, df], axis = 1)
    return income_df

In [14]:
income_df_go()

,2010-Illinois,2010-Cook County,2011-Illinois,2011-Cook County,2012-Illinois,2012-Cook County,2013-Illinois,2013-Cook County,2014-Illinois,2014-Cook County,2015-Illinois,2015-Cook County,2016-Illinois,2016-Cook County,2017-Illinois,2017-Cook County,2018-Illinois,2018-Cook County,2019-Illinois,2019-Cook County
Year,2010,2010,2011,2011,2012,2012,2013,2013,2014,2014,2015,2015,2016,2016,2017,2017,2018,2018,2019,2019
id,0400000US17,0500000US17031,0400000US17,0500000US17031,0400000US17,0500000US17031,0400000US17,0500000US17031,0400000US17,0500000US17031,0400000US17,0500000US17031,0400000US17,0500000US17031,0400000US17,0500000US17031,0400000US17,0500000US17031,0400000US17,0500000US17031
Geographic Area Name,Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois",Illinois,"Cook County, Illinois"
Households!!Estimate!!Total,4752857,1922125,4737208,1916910,4770194,1936001,4783421,1939894,4772421,1938055,4794523,1954712,4822046,1963973,4808672,1948373,4864864,1981796,4866006,1985613
"Households!!Estimate!!Less than $10,000",7.1,8.6,7.6,9.0,7.5,9.1,7.7,9.2,7.3,8.6,6.8,8.1,7.0,8.3,6.7,7.8,6.4,7.3,5.9,7.0
"Households!!Estimate!!$10,000 to $14,999",4.9,5.1,5.0,5.2,4.9,5.5,4.7,5.1,4.6,4.9,4.3,4.7,4.3,4.5,4.1,4.2,3.8,3.9,3.7,4.0
"Households!!Estimate!!$15,000 to $24,999",11.0,11.5,11.1,11.4,10.4,10.6,9.9,10.3,9.9,10.6,9.8,10.5,9.3,9.5,8.9,9.4,8.8,9.2,8.0,7.9
"Households!!Estimate!!$25,000 to $34,999",10.2,9.8,10.0,10.3,9.7,9.7,9.7,9.5,9.3,9.1,9.1,9.2,8.8,8.7,8.4,8.5,8.5,8.5,8.0,8.2
"Households!!Estimate!!$35,000 to $49,999",13.8,13.5,13.2,13.3,13.2,13.1,13.0,12.6,12.8,12.4,12.6,11.9,12.3,11.7,11.8,11.9,11.6,11.5,11.4,10.5
"Households!!Estimate!!$50,000 to $74,999",18.4,17.7,18.4,17.7,17.9,16.9,17.9,17.2,17.8,17.1,17.5,16.1,17.1,16.5,17.4,16.5,17.2,16.4,16.5,15.7


## Read Data Using API

In [15]:
# references 
# https://towardsdatascience.com/mapping-us-census-data-with-python-607df3de4b9c
# https://pypi.org/project/CensusData/
# https://towardsdatascience.com/using-the-us-census-api-for-data-analysis-a-beginners-guide-98063791785c

# function for retrieving population data
def get_acs5_county_population(year):
    data = censusdata.download('acs5', year, censusdata.censusgeo([('county', '*')]),
                               ['B05002_001E', 'B05002_002E', 'B05002_003E', 'B05002_004E', 'B05002_009E', 'B05002_013E'])
    
    # create dictionary for the column names
    column_names = {'B05002_001E': 'total_population', 'B05002_002E': 'total_native', 
                    'B05002_003E': 'total_born_in_state', 'B05002_004E': 'total_born_out_state',
                    'B05002_009E': 'total_born_outside_US', 'B05002_013E': 'total_foreign_born'}
    
    # create new column for county name and state name, the index would be FIPS codes 
    new_indices = []
    county_names = []
    county_ids = []
    state_names = []
    state_ids = []
    
    
    for index in data.index.tolist():
        new_index = index.geo[0][1] + index.geo[1][1]
        new_indices.append(new_index)
        county_name = index.name.split(',')[0]
        county_names.append(county_name)
        state_name = index.name.split(',')[1]
        state_names.append(state_name)
        state_id = index.geo[0][1]
        state_ids.append(state_id)
        county_id = index.geo[1][1]
        county_ids.append(county_id)

    data.index = new_indices
    data['county_name'] = county_names
    data['state_name'] = state_names
    data['county_id'] = county_ids
    data['state_id'] =  state_ids
    data['year'] =  year
    
    # replace column names 
    data = data[['year', 'state_name', 'county_name', 'state_id', 'county_id', 'B05002_001E',
                 'B05002_002E', 'B05002_003E', 'B05002_004E', 'B05002_009E', 'B05002_013E']]
    data = data.rename(column_names, axis=1) 
    return data

In [16]:
# function for retrieving income data
def get_acs5_county_income(year):
    data = censusdata.download('acs5', year, censusdata.censusgeo([('county', '*')]), ['B19301_001E'])
    
    # create dictionary for the column names
    column_names = {'B19301_001E': 'income_past12m'}
    
    # create new column for county name and state name, the index would be FIPS codes 
    new_indices = []
    county_names = []
    county_ids = []
    state_names = []
    state_ids = []
    
    for index in data.index.tolist():
        new_index = index.geo[0][1] + index.geo[1][1]
        new_indices.append(new_index)
        county_name = index.name.split(',')[0]
        county_names.append(county_name)
        state_name = index.name.split(',')[1]
        state_names.append(state_name)
        state_id = index.geo[0][1]
        state_ids.append(state_id)
        county_id = index.geo[1][1]
        county_ids.append(county_id)

    data.index = new_indices
    data['county_name'] = county_names
    data['state_name'] = state_names
    data['county_id'] = county_ids
    data['state_id'] =  state_ids
    data['year'] =  year
    
    # replace column names 
    data = data[['year', 'state_name', 'county_name', 'state_id', 'county_id', 'B19301_001E']]
    data = data.rename(column_names, axis=1) 
    return data

In [17]:
population_2010 = get_acs5_county_population(2010)
population_2011 = get_acs5_county_population(2011)
population_2012 = get_acs5_county_population(2012)
population_2013 = get_acs5_county_population(2013)
population_2014 = get_acs5_county_population(2014)
population_2015 = get_acs5_county_population(2015)
population_2016 = get_acs5_county_population(2016)
population_2017 = get_acs5_county_population(2017)
population_2018 = get_acs5_county_population(2018)
population_2019 = get_acs5_county_population(2019)

In [18]:
years = list(range(2010, 2020))
print(years)

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [19]:
df_population = pd.DataFrame()

for year in years: 
    population_year = get_acs5_county_population(year) 
    df_population = df_population.append(population_year)

In [20]:
df_population.reset_index(drop = True, inplace = True)

,year,state_name,county_name,state_id,county_id,total_population,total_native,total_born_in_state,total_born_out_state,total_born_outside_US,total_foreign_born
0,2010,Puerto Rico,Manatí Municipio,72,091,44631,44368,0,2638,41730,263
1,2010,California,Merced County,06,047,250699,188498,156434,29768,2296,62201
2,2010,California,Modoc County,06,049,9605,8971,6067,2882,22,634
3,2010,California,Mono County,06,051,13905,11341,7059,4108,174,2564
4,2010,California,Monterey County,06,053,407435,284647,208218,69900,6529,122788
...,...,...,...,...,...,...,...,...,...,...,...
32199,2019,Tennessee,Crockett County,47,033,14399,13774,11231,2490,53,625
32200,2019,Tennessee,Lake County,47,095,7401,7307,5436,1839,32,94
32201,2019,Tennessee,Knox County,47,093,461104,438362,272843,161946,3573,22742
32202,2019,Washington,Benton County,53,005,197518,176385,104182,70549,1654,21133


In [21]:
illinois_population = df_population[df_population['state_id'] == '17']

In [22]:
illinois_population

,year,state_name,county_name,state_id,county_id,total_population,total_native,total_born_in_state,total_born_out_state,total_born_outside_US,total_foreign_born
17001,2010,Illinois,Adams County,17,001,67030,66319,52137,13948,234,711
17003,2010,Illinois,Alexander County,17,003,8449,8292,4896,3335,61,157
17005,2010,Illinois,Bond County,17,005,17904,17700,13565,4092,43,204
17007,2010,Illinois,Boone County,17,007,53567,48152,36708,11088,356,5415
17009,2010,Illinois,Brown County,17,009,6897,6711,5943,739,29,186
...,...,...,...,...,...,...,...,...,...,...,...
17147,2019,Illinois,Piatt County,17,147,16401,16190,13563,2539,88,211
17175,2019,Illinois,Stark County,17,175,5447,5384,4677,687,20,63
17005,2019,Illinois,Bond County,17,005,16589,16228,12328,3855,45,361
17015,2019,Illinois,Carroll County,17,015,14466,14185,9810,4313,62,281


In [55]:
income_2010 = get_acs5_county_income(2010)
income_2015 = get_acs5_county_income(2015)
income_2019 = get_acs5_county_income(2019)

In [56]:
df_income = pd.DataFrame()

for year in years: 
    income_year = get_acs5_county_income(year) 
    df_income = df_income.append(income_year)
    
df_income

,year,state_name,county_name,state_id,county_id,income_past12m
72091,2010,Puerto Rico,Manatí Municipio,72,091,8949
06047,2010,California,Merced County,06,047,18041
06049,2010,California,Modoc County,06,049,20536
06051,2010,California,Mono County,06,051,27321
06053,2010,California,Monterey County,06,053,25776
...,...,...,...,...,...,...
47033,2019,Tennessee,Crockett County,47,033,23771
47095,2019,Tennessee,Lake County,47,095,15732
47093,2019,Tennessee,Knox County,47,093,33229
53005,2019,Washington,Benton County,53,005,32882


In [25]:
population_2010[population_2010['county_name'] == 'Clay County']

,year,state_name,county_name,state_id,county_id,total_population,total_native,total_born_in_state,total_born_out_state,total_born_outside_US,total_foreign_born
12019,2010,Florida,Clay County,12,019,186076,175187,77817,92531,4839,10889
13061,2010,Georgia,Clay County,13,061,2981,2981,2393,584,4,0
01027,2010,Alabama,Clay County,01,027,14040,13754,11421,2299,34,286
05021,2010,Arkansas,Clay County,05,021,16297,16228,9500,6705,23,69
19041,2010,Iowa,Clay County,19,041,16718,16380,13058,3281,41,338
20027,2010,Kansas,Clay County,20,027,8531,8422,5908,2406,108,109
21051,2010,Kentucky,Clay County,21,051,22197,21952,19292,2652,8,245
27027,2010,Minnesota,Clay County,27,027,57089,55197,24533,30402,262,1892
28025,2010,Mississippi,Clay County,28,025,20883,20834,17778,3026,30,49
17025,2010,Illinois,Clay County,17,025,13927,13814,11988,1807,19,113


In [54]:
df_income.reset_index(drop = True, inplace = True)
df_income

,year,state_name,county_name,state_id,county_id,income_past12m
0,2010,Puerto Rico,Manatí Municipio,72,091,8949
1,2010,California,Merced County,06,047,18041
2,2010,California,Modoc County,06,049,20536
3,2010,California,Mono County,06,051,27321
4,2010,California,Monterey County,06,053,25776
...,...,...,...,...,...,...
32199,2019,Tennessee,Crockett County,47,033,23771
32200,2019,Tennessee,Lake County,47,095,15732
32201,2019,Tennessee,Knox County,47,093,33229
32202,2019,Washington,Benton County,53,005,32882


## Read CSV Files

In [28]:
# https://developer.ipums.org/docs/workflows/create_extracts/nhgis_data/
# The API might not be available for vintage data, hence we downloaded the csv files and read those instead
# Read NHGIS IPUMS data for population 
# turns out the table is slightly different so I couldn't generalized 
# but here is the generalized funciton that I tried 

In [29]:
def get_nhgis_county_1990(path, folder, fname):
    fname = os.path.join(path, folder, fname)
    data = pd.read_csv(fname)
    
    data = data.drop(['ANRCA', 'AIANHHA', 'RES_ONLYA', 'TRUSTA',
                      'RES_TRSTA', 'BLCK_GRPA', 'TRACTA', 'CD101A',
                      'C_CITYA', 'CTY_SUBA', 'PLACEA', 'PMSAA',
                      'URB_AREAA', 'ZIPA', 'CD103A', 'COUNTY',
                      'DIVISIONA', 'MSA_CMSAA', 'REGIONA', 'URBRURALA'], axis=1)
    
    data = data.iloc[2:]
    data = data.reset_index(drop = True)
    data['index'] = data['STATEA'] + data['COUNTYA']
    
    return data

In [30]:
def get_nhgis_county_2000(path, folder, fname):
    fname = os.path.join(path, folder, fname)
    data = pd.read_csv(fname)
    
    data = data.drop(['REGIONA', 'DIVISIONA', 'CTY_SUBA', 'PLACEA',
                      'CD108A', 'CD109A', 'ZIP3A', 'ZCTAA', 'TRACTA',
                      'TRBL_CTA', 'BLCK_GRPA', 'TRBL_BGA', 'C_CITYA', 
                      'AIANHHA', 'RES_ONLYA', 'TRUSTA', 'NECMAA', 
                      'TRBL_SUBA', 'ANRCA', 'URB_AREAA', 'URBRURALA', 
                      'CD106A', 'MSA_CMSAA', 'COUNTY', 'PMSAA'], axis=1)
    
    data = data.iloc[2:]
    data = data.reset_index(drop = True)
    data['index'] = data['STATEA'] + data['COUNTYA']
    
    return data

In [31]:
# income data 1990
income_1990_data = get_nhgis_county_1990(path, 'IPUMS NHGIS', 'income_1990_county.csv')

names_income_1990 = {'E01001': 'per_capita_income'}
income_1990_data = income_1990_data.rename(names_income_1990, axis=1)

income_1990_data.head()

,GISJOIN,YEAR,COUNTYA,STATE,STATEA,ANPSADPI,per_capita_income,index
0,G0100030,1990,003,Alabama,01,Baldwin County,12275,01003
1,G0100050,1990,005,Alabama,01,Barbour County,9515,01005
2,G0100070,1990,007,Alabama,01,Bibb County,8973,01007
3,G0100090,1990,009,Alabama,01,Blount County,10168,01009
4,G0100110,1990,011,Alabama,01,Bullock County,6922,01011


In [32]:
# population data 1990
population_1990_data = get_nhgis_county_1990(path, 'IPUMS NHGIS', 'nativity_1990_county.csv')

population_1990_data.iloc[:,6:] = population_1990_data.iloc[:,6:].astype(float)

population_1990_data['total_born_out_state'] = population_1990_data['E3N002'] + population_1990_data['E3N003'] + population_1990_data['E3N004'] + population_1990_data['E3N005']
population_1990_data['total_born_outside_US'] = population_1990_data['E3N006'] + population_1990_data['E3N007'] + population_1990_data['E3N008'] 

population_1990_data = population_1990_data.drop(['E3N002', 'E3N003', 'E3N004', 'E3N005', 'E3N006',
                                                  'E3N007', 'E3N008'], axis=1)

names_population_1990 = {'E3N001': 'total_born_in_state', 'E3N009': 'total_foreign_born'}
population_1990_data = population_1990_data.rename(names_population_1990, axis=1) 

population_1990_data.head()

,GISJOIN,YEAR,COUNTYA,STATE,STATEA,ANPSADPI,total_born_in_state,total_foreign_born,index,total_born_out_state,total_born_outside_US
0,G0100030,1990,003,Alabama,01,Baldwin County,63928,975,1003,32789,588
1,G0100050,1990,005,Alabama,01,Barbour County,19237,82,1005,6022,76
2,G0100070,1990,007,Alabama,01,Bibb County,14775,18,1007,1759,24
3,G0100090,1990,009,Alabama,01,Blount County,33869,247,1009,5043,89
4,G0100110,1990,011,Alabama,01,Bullock County,9922,24,1011,1096,0


In [33]:
# income data 2000
income_2000_data = get_nhgis_county_2000(path, 'IPUMS NHGIS', 'income_2000_county.csv')

names_income_2000 = {'GNW001': 'per_capita_income'}
income_2000_data = income_2000_data.rename(names_income_2000, axis=1) 

income_2000_data.head()

,GISJOIN,YEAR,STATE,STATEA,COUNTYA,NAME,per_capita_income,index
0,G0100030,2000,Alabama,01,003,Baldwin County,20826,01003
1,G0100050,2000,Alabama,01,005,Barbour County,13316,01005
2,G0100070,2000,Alabama,01,007,Bibb County,14105,01007
3,G0100090,2000,Alabama,01,009,Blount County,16325,01009
4,G0100110,2000,Alabama,01,011,Bullock County,10163,01011


In [34]:
# population data 2000
population_2000_data = get_nhgis_county_2000(path, 'IPUMS NHGIS', 'nativity_2000_county.csv')

names_population_2000 = {'GI8001': 'total_native', 'GI9001': 'total_born_in_state', 
                         'GI9002': 'total_born_out_state', 'GI9003': 'total_born_outside_US', 
                         'GI8002': 'total_foreign_born'}

population_2000_data = population_2000_data.rename(names_population_2000, axis=1) 
population_2000_data.head()

,GISJOIN,YEAR,STATE,STATEA,COUNTYA,NAME,total_native,total_foreign_born,total_born_in_state,total_born_out_state,total_born_outside_US,index
0,G0100030,2000,Alabama,01,003,Baldwin County,137458,2957,81709,54637,1112,01003
1,G0100050,2000,Alabama,01,005,Barbour County,28603,435,21707,6826,70,01005
2,G0100070,2000,Alabama,01,007,Bibb County,20737,89,18303,2387,47,01007
3,G0100090,2000,Alabama,01,009,Blount County,49443,1581,41300,7893,250,01009
4,G0100110,2000,Alabama,01,011,Bullock County,11356,358,10035,1291,30,01011


In [35]:
# Read Universities R&D Data based on Names 
rd = os.path.join(path, 'NCSES', 'HERD_data.csv')
rd_data = pd.read_csv(rd, skiprows=10)

rd_data = rd_data.iloc[2:,:-1]
column_names = {'Unnamed: 0': 'state_name', '<Fiscal Year>': 'institution_name'}
rd_data = rd_data.rename(column_names, axis=1) 
rd_data = rd_data[rd_data['institution_name'].str.contains('Total for selected values')==False]
rd_data.iloc[:,2:] = rd_data.iloc[:,2:].replace({'-':np.nan}).replace(r',','',regex=True)
rd_data.iloc[:,2:] = rd_data.iloc[:,2:].astype(float)
rd_data = rd_data.sort_values(['2010'], ascending = False)

#rd_data  = rd_data.nlargest(20,'2010')
rd_data = rd_data.head(50)

rd_data = rd_data.reset_index(drop = True)

In [36]:
rd_data

,state_name,institution_name,2019,2018,2017,2016,2015,2014,2013,2012,...,1982,1981,1980,1979,1978,1977,1976,1975,1974,1973
0,Maryland,Johns Hopkins U.,2597238,2416268,2373646,2294982,2200312,2162885,2131165,2106185,...,590129,584560,598977,613165,182257,184941,181944,189191,186417,168645
1,Michigan,"U. Michigan, Ann Arbor",1491880,1453619,1417476,1355976,1306704,1301372,1351399,1322711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wisconsin,U. Wisconsin-Madison,1154945,1094633,1105543,1092825,1020221,1069218,1104123,1169779,...,320608,318840,326915,315224,301500,299632,296227,319732,315142,335953
3,Washington,"U. Washington, Seattle",1269137,1283849,1248951,1206102,1126613,1134588,1171945,1109008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,North Carolina,Duke U.,1091903,1060213,1043949,996632,989322,1000013,975697,1009911,...,101065,100291,92414,95395,93549,90745,90464,89756,77636,88819
5,California,"U. California, San Diego",1205183,1148822,1049998,1026215,1051130,1029503,1057003,1073864,...,282698,297877,295297,277920,266046,262011,261208,257626,246031,266481
6,California,"U. California, Los Angeles",1162997,1196868,997624,979404,974558,914543,949986,1003375,...,210239,210307,210381,194727,201893,207122,199400,193747,183093,208636
7,California,"U. California, San Francisco",1420031,1448954,1305625,1221755,1075135,1045555,1024854,1032673,...,193932,192791,173696,157699,158053,148523,151866,144934,124178,116142
8,California,Stanford U.,1071960,1051120,1028001,1006535,975822,925200,929143,903238,...,277548,280582,267596,262266,246359,230269,230939,224050,197494,215391
9,Pennsylvania,U. Pennsylvania,1340965,1309300,1273105,1223801,824581,798949,814134,847077,...,229661,225685,222834,211402,213664,211439,221168,206461,193486,161966


In [37]:
rd_data.state_name.unique()

array(['Maryland', 'Michigan', 'Wisconsin', 'Washington',
       'North Carolina', 'California', 'Pennsylvania', 'New York',
       'Minnesota', 'Ohio', 'Missouri', 'Texas', 'Florida',
       'Massachusetts', 'Connecticut', 'Georgia', 'Arizona', 'Illinois',
       'Indiana', 'Tennessee', 'Alabama', 'Iowa', 'New Jersey',
       'Virginia', 'Colorado'], dtype=object)

In [38]:
# Read Universities R&D Data based on ID 
# # https://stackoverflow.com/questions/45416684/python-pandas-replace-multiple-columns-zero-to-nan
rd_id = os.path.join(path, 'NCSES', 'HERD_data_IPEDS.csv')
rd_id_data = pd.read_csv(rd_id, skiprows=10)


rd_id_data = rd_id_data.iloc[2:]
column_names_id = {'Unnamed: 0': 'state_name', '<Fiscal Year>': 'IPEDSID'}
rd_id_data = rd_id_data.rename(column_names_id, axis=1) 
rd_id_data = rd_id_data[rd_id_data['IPEDSID'].str.contains('Total for selected values') == False]
rd_id_data = rd_id_data[rd_id_data['IPEDSID'].str.contains('No match or exact match for IPEDS UnitID') == False]
rd_id_data.iloc[:,2:] = rd_id_data.iloc[:,2:].replace({'-':np.nan}).replace(r',','',regex=True)
rd_id_data.iloc[:,2:] = rd_id_data.iloc[:,2:].astype(float)
rd_id_data  = rd_id_data.sort_values(['2010'], ascending = False)

rd_id_data = rd_id_data.head(50)

rd_id_data = rd_id_data.reset_index(drop = True)

In [39]:
rd_id_data

,state_name,IPEDSID,2019,2018,2017,2016,2015,2014,2013,2012,...,1982,1981,1980,1979,1978,1977,1976,1975,1974,1973
0,Maryland,162928,2597238,2416268,2373646,2294982,2200312,2162885,2131165,2106185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Michigan,170976,1491880,1453619,1417476,1355976,1306704,1301372,1351399,1322711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wisconsin,240444,1154945,1094633,1105543,1092825,1020221,1069218,1104123,1169779,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Washington,236948,1269137,1283849,1248951,1206102,1126613,1134588,1171945,1109008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,North Carolina,198419,1091903,1060213,1043949,996632,989322,1000013,975697,1009911,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,California,110680,1205183,1148822,1049998,1026215,1051130,1029503,1057003,1073864,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,California,110662,1162997,1196868,997624,979404,974558,914543,949986,1003375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,California,110699,1420031,1448954,1305625,1221755,1075135,1045555,1024854,1032673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,California,243744,1071960,1051120,1028001,1006535,975822,925200,929143,903238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Pennsylvania,215062,1340965,1309300,1273105,1223801,824581,798949,814134,847077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
rd_id_data.state_name.unique()

array(['Maryland', 'Michigan', 'Wisconsin', 'Washington',
       'North Carolina', 'California', 'Pennsylvania', 'New York',
       'Minnesota', 'Missouri', 'Texas', 'Florida', 'Massachusetts',
       'Connecticut', 'Georgia', 'Arizona', 'Illinois', 'Indiana',
       'Tennessee', 'Alabama', 'Iowa', 'New Jersey', 'Ohio', 'Virginia',
       'Colorado', 'Utah'], dtype=object)

In [41]:
rd_data = rd_data.iloc[2:,:-1]
column_names = {'Unnamed: 0': 'state_name', '<Fiscal Year>': 'institution_name'}
rd_data = rd_data.rename(column_names, axis=1) 
rd_data = rd_data[rd_data['institution_name'].str.contains('Total for selected values')==False]
rd_data = rd_data.reset_index(drop = True)

rd_data['2010'] = rd_data['2010'].replace(',','')
#rd_data['2010'].astype(float)
#a = rd_data.sort_values(by=['2010'], ascending = True)
#a[['state_name', 'institution_name','2010']]

In [42]:
# select column that align with the year that we are
rd_data_filter = rd_data[['state_name', 'institution_name', '2019', '2015', '2010', '2000', '1990']]
rd_id_data_filter = rd_id_data[['state_name', 'IPEDSID', '2019', '2015', '2010', '2000', '1990']]

In [43]:
rd_data_filter

,state_name,institution_name,2019,2015,2010,2000,1990
0,Wisconsin,U. Wisconsin-Madison,1154945,1020221,1070948,710018,486629
1,Washington,"U. Washington, Seattle",1269137,1126613,1064128,NaN,NaN
2,North Carolina,Duke U.,1091903,989322,1023081,456760,220993
3,California,"U. California, San Diego",1205183,1051130,981389,664163,372277
4,California,"U. California, Los Angeles",1162997,974558,974913,679874,387610
5,California,"U. California, San Francisco",1420031,1075135,973367,567405,374234
6,California,Stanford U.,1071960,975822,873826,586372,470405
7,Pennsylvania,U. Pennsylvania,1340965,824581,870166,551236,297451
8,Pennsylvania,"U. Pittsburgh, Pittsburgh",962313,821849,855776,377587,184882
9,New York,Columbia U. in the City of New York,893465,828485,839902,409458,287053


In [44]:
rd_id_data_filter 

,state_name,IPEDSID,2019,2015,2010,2000,1990
0,Maryland,162928,2597238,2200312,2085599,NaN,NaN
1,Michigan,170976,1491880,1306704,1232377,NaN,NaN
2,Wisconsin,240444,1154945,1020221,1070948,NaN,NaN
3,Washington,236948,1269137,1126613,1064128,NaN,NaN
4,North Carolina,198419,1091903,989322,1023081,NaN,NaN
5,California,110680,1205183,1051130,981389,NaN,NaN
6,California,110662,1162997,974558,974913,NaN,NaN
7,California,110699,1420031,1075135,973367,NaN,NaN
8,California,243744,1071960,975822,873826,NaN,NaN
9,Pennsylvania,215062,1340965,824581,870166,NaN,NaN


## Load Shape File 

In [45]:
# merge function
def merge(df1, df2, left_on, right_on):
    merge = df1.merge(df2, left_on=left_on, right_on=right_on, how='left')
    
    return merge

In [46]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html

county_shp = os.path.join(path, 'cb_2020_us_county_20m', 'cb_2020_us_county_20m.shp')
county = geopandas.read_file(county_shp)

In [47]:
# making the plot including only 48 states within NA
contiguous_fips = [state.fips for state in us.STATES_CONTIGUOUS]
mask_fips = county['STATEFP'].isin(contiguous_fips)
county_mask = county[mask_fips]

In [48]:
county_mask

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,01,061,00161556,0500000US01061,01061,Geneva,Geneva County,AL,Alabama,06,1487908432,11567409,"POLYGON ((-86.19348 31.19221, -86.12541 31.182..."
1,08,125,00198178,0500000US08125,08125,Yuma,Yuma County,CO,Colorado,06,6123763559,11134665,"POLYGON ((-102.80377 40.00255, -102.79358 40.3..."
2,17,177,01785076,0500000US17177,17177,Stephenson,Stephenson County,IL,Illinois,06,1461392061,1350223,"POLYGON ((-89.92647 42.50579, -89.83759 42.504..."
3,28,153,00695797,0500000US28153,28153,Wayne,Wayne County,MS,Mississippi,06,2099745602,7255476,"POLYGON ((-88.94335 31.82456, -88.91046 31.826..."
4,34,041,00882237,0500000US34041,34041,Warren,Warren County,NJ,New Jersey,06,923435921,15822933,"POLYGON ((-75.19261 40.71587, -75.17748 40.764..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,30,091,01677577,0500000US30091,30091,Sheridan,Sheridan County,MT,Montana,06,4343256743,75111952,"POLYGON ((-105.05763 48.99923, -104.87553 48.9..."
3217,54,047,01689162,0500000US54047,54047,McDowell,McDowell County,WV,West Virginia,06,1381648569,3731571,"POLYGON ((-81.98489 37.45432, -81.93228 37.511..."
3218,54,031,01718557,0500000US54031,54031,Hardy,Hardy County,WV,West Virginia,06,1508190150,5624064,"POLYGON ((-79.13430 38.81334, -79.04685 38.927..."
3219,54,091,01689856,0500000US54091,54091,Taylor,Taylor County,WV,West Virginia,06,447475053,7532835,"POLYGON ((-80.19729 39.39319, -80.08733 39.386..."


In [49]:
illinois = county[county['STUSPS'] == 'IL']

In [50]:
illinois

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
2,17,177,01785076,0500000US17177,17177,Stephenson,Stephenson County,IL,Illinois,06,1461392061,1350223,"POLYGON ((-89.92647 42.50579, -89.83759 42.504..."
11,17,155,00424279,0500000US17155,17155,Putnam,Putnam County,IL,Illinois,06,414649315,31195045,"POLYGON ((-89.46639 41.23387, -89.35673 41.233..."
105,17,159,00424281,0500000US17159,17159,Richland,Richland County,IL,Illinois,06,932484098,4874777,"POLYGON ((-88.25778 38.73114, -88.25858 38.847..."
136,17,095,00424249,0500000US17095,17095,Knox,Knox County,IL,Illinois,06,1855427713,8837382,"POLYGON ((-90.43942 41.06399, -90.43765 41.151..."
231,17,029,00424216,0500000US17029,17029,Coles,Coles County,IL,Illinois,06,1316418028,4650074,"POLYGON ((-88.47207 39.65159, -88.06352 39.652..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3056,17,059,00424231,0500000US17059,17059,Gallatin,Gallatin County,IL,Illinois,06,836413648,13357002,"POLYGON ((-88.37453 37.90768, -88.37452 37.908..."
3109,17,043,00422191,0500000US17043,17043,DuPage,DuPage County,IL,Illinois,06,848995066,22247645,"POLYGON ((-88.26287 41.98638, -88.17410 41.987..."
3110,17,065,00424234,0500000US17065,17065,Hamilton,Hamilton County,IL,Illinois,06,1125731578,3172088,"POLYGON ((-88.70461 38.12519, -88.70239 38.256..."
3111,17,067,00424235,0500000US17067,17067,Hancock,Hancock County,IL,Illinois,06,2055798688,53563362,"POLYGON ((-91.49696 40.24870, -91.49289 40.269..."


In [51]:
illinois_population_2019 

NameError: name 'illinois_population_2019' is not defined

In [ ]:
illinois.merge(illinois_population_2019, left_on='COUNTYFP', right_on='county_id')

In [ ]:
fig, ax = plt.subplots(figsize=(80,80))

ax = illinois.plot(ax=ax, color='white', edgecolor='black')

ax.axis('off');

In [ ]:
fig, ax = plt.subplots(figsize=(150,150))

ax = county_mask.plot(ax=ax, color='white', edgecolor='black')

ax.axis('off');

In [ ]:
universities_shp = os.path.join(path, 'Colleges_and_Universities-shp', 'Colleges_and_Universities.shp')
universities = geopandas.read_file(universities_shp)

In [ ]:
universities[universities['NAME'] == 'University of Michigan-Ann Arbor'.upper()]

In [ ]:
universities.insert(6,'fips', universities['STATE'].map(us.states.mapping('abbr','fips')))
mask_contiguous_universities = universities['fips'].isin(contiguous_fips)
universities_mask = universities[mask_contiguous_universities]

In [ ]:
fig, ax = plt.subplots(figsize=(150,150))

ax = universities_mask.plot(ax=ax, color='white', edgecolor='black')

ax.axis('off');

## Simple Linear Regression Example

In [ ]:
iris = sns.load_dataset('iris')
iris.head()